## CPSC 439 (Spring 2022) Project 1: Turing Machines

###  Group Members
* Bradley Diep
* John Dinh
* Jason Duong
* Omid Nikjoo

### Documentation
1. **DFA Representation**
    <center>$RE = (1^+01^+01^+0)^*0(1^+0)^*$</center>

    * Each transition is represented as a tuple $(a, b, c)$ encoded as $1^a01^b01^c$ for $a, b, c \in N$
    * Each final state, $d$, is represented as $1^d0$ for $d\in N$
    * The $00$ token string deliminates objects T and S
    *_Example_: ``10110111001101 = T: (1, 11, 111), S: {11, 1} = T: (q0, q1, q2), S: {q1, q0}``

# Hints
* can use any data structure, but still need {0, 1}*

In [84]:
from collections import namedtuple

DFA = namedtuple('DFA', ['T', 'S'])
q0, q1, q2, q3 = [*range(4)]

In [85]:
def encode(M):
    '''converts DFA model to binary representation'''
    code = ''

    for (current, symbol), successor in M.T.items():
        code += ''.join(list(map(lambda offset: '1' * (offset + 1) + '0',
                                 [current, symbol, successor])))
    
    code += '0'
    
    for final in M.S:
        code += ''.join(list(map(lambda offset: '1' * (offset + 1) + '0', [final])))

    return code

# DFA Representation
DFA.__repr__ = encode

In [93]:
from math import log2, ceil

def encode(*re):
    '''encodes a regular expression'''
    alphabet = r'01(◦|*)∅'
    eval = {exp: f'{value:0{ceil(log2(len(alphabet)))}b}' \
            for value, exp in enumerate(list(alphabet))}
 
    return '_'.join([''.join([eval[i] for i in k]) \
                    for j, k in enumerate(re)])

XOR = r'0*(10*10*)*10*'
F = r'(010)*'

print('RE:', encode(F, '010010'))

RE: 010000001000110101_000001000000001000


In [94]:
'''Figure 6.3'''

# Transition Fucnction
δ = dict([ 
    ((q0, 0), q0), ((q0, 1), q1),
    ((q1, 0), q1), ((q1, 1), q0),
])

XOR_DFA = DFA(δ, {q1})
print('XOR_DFA:', XOR_DFA)

XOR_DFA: 1010101011011011010110110110100110


In [95]:
'''Figure 6.4'''

# Transition Function
δ = dict([
    ((q0, 0), q2), ((q0, 1), q3),
    ((q1, 0), q0), ((q1, 1), q3),
    ((q2, 0), q3), ((q2, 1), q1),
    ((q3, 0), q3), ((q3, 1), q3),
])

F_DFA = DFA(δ, {q0})
print('F_DFA:', F_DFA)

F_DFA: 1010111010110111101101010110110111101110101111011101101101111010111101111011011110010


In [101]:
class TuringMachine():
    '''TM Implementation'''
    alphabet = ['0', '1', '▷', '∅', '🔥']
    states = ['Q' + str(i) for i in range(6)] + ['QACCEPT', 'QREJECT']
    
    def __init__(self, source):
        self._input = source
        
        self._head = 0
        self._state = self.states[0]
        self._tape = ['▷'] + [i for i in self._input] + ['∅']


    def __str__(self):
        return ''.join([x for x in self._tape])
    
    def debug(self):
        print(f'{self._state:12} {self._tape[self._head]:5}', end='')

    def transition(self, state, symbol):
        action = 'R'
        return state, symbol, action
        
    def run(self):
        '''computes a function given an input'''
        print(self.states)
        self.debug()
        return None

        while True:
            self.debug()
            
            self._state, self._tape[self._head], action = \
                self.transition(self._state, self._tape[self._head])
        
            self.debug()
            print(action)

            if self._head == len(self._tape) - 1:
                self._tape.append('∅')
            
            if action == 'L':
                self._head = max(0, self._head - 1)
            elif action == 'R':
                self._head += 1
            elif action == 'S':
                continue
            elif action == 'H':
                break

        return ''.join(self._tape[self._tape.index('▷') + 1 :
                                  self._tape.index('∅')])


M = TuringMachine(encode(F, '1011'))

print(str(M))
M.run()

▷010000001000110101_001000001001∅
['Q0', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'QACCEPT', 'QREJECT']
Q0           ▷    

In [104]:
M = XOR_DFA

alphabet = {0, 1}
state = sorted(list(set(([x[0] for x in M.T.keys()]))))
current = state[0]
final = M.S

print(state, alphabet, current, final, sep=', ')

[0, 1], {0, 1}, 0, {1}
